In [ ]:
# Objective: Getting pop, buildings and area per landslide risk zone

# Input: admin0, admin1, region, landslide_raster (with 8 different risk categories)
# Output: admin0, admin1 and region with pop, buildings and area per risk category per admin polygon

# Steps:

## Pop
# Load population data and landslide data
# Reproject so pixel size matche between pop and landslide
# Mask pop with land slide categories
# Apply zonal stats across masked population dataset

## Area
# Get count of none-masked pixels from population data above for each risk category (already masked by risk category)
# Get pixel size
# Multiply pixel size with count

## Buildings
# Extract pixel from landslide to each building centroid (using zonal stats on point)
# Join buildings and admin polygons
# Group by each risk category for each polygon

## Aggregate all columns to 5 risk categories
## Join final columns to adm_stats original

In [2]:
import os
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import rioxarray as rxr
from rasterio.crs import CRS
from sqlalchemy import create_engine
import rasterstats
from shapely.ops import transform
from datetime import datetime
import numpy as np

from rasterio.enums import Resampling

import xarray
import matplotlib.pyplot as plt

In [3]:
pcode = 'reg_pcode'

In [4]:
os.chdir('D:\iMMAP\proj\ASDC\data\Landslide\process_v09')

In [5]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [6]:
pd.set_option('display.max_columns', None)

### Select input data

In [7]:
print('Loading adm')
adm = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_region_stats', con) #.to_crs(repro_crs)
print('Loading landslide')
landslide = xarray.open_dataarray(r'D:\iMMAP\data\Afghanistan\HSDC-Official\afg_ls_30m_wb_s3.tif').squeeze() # D:\iMMAP\proj\ASDC\data\afg_ls_30m_wb_s3_compressed.tif
print('Loading pop')
pop = xarray.open_dataarray(r'D:\iMMAP\data\Afghanistan\HSDC-Official\afg_worldpop_2020_UNadj_unconstrained.tif').squeeze() # D:\iMMAP\data\Afghanistan\HSDC-Official\afg_worldpop_2020_UNadj_unconstrained.tif
print('Loading build')
build = gpd.GeoDataFrame.from_postgis('SELECT * from afg_buildings_microsoft_centroids', con) #.to_crs(repro_crs)
#adm_stats_original = gpd.GeoDataFrame.from_postgis('SELECT * FROM afg_admbnda_adm1_stats', con)

Loading adm


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "afg_admbnda_region_stats" does not exist
LINE 1: SELECT * from afg_admbnda_region_stats
                      ^

[SQL: SELECT * from afg_admbnda_region_stats]
(Background on this error at: https://sqlalche.me/e/14/f405)

## 1. Population

### 1.1. Reproject landslide to match pop

In [ ]:
# Define function to describe raster
def print_raster(raster):
    print(
        f"shape: {raster.rio.shape}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: {raster.rio.bounds()}\n"
        f"sum: {raster.sum().item()}\n"
        f"CRS: {raster.rio.crs}\n"
    )

In [ ]:
print("Original Raster:\n----------------\n")
print_raster(landslide)
print("Raster to Match:\n----------------\n")
print_raster(pop)

In [ ]:
# Reproject to match target raster
landslide_repro = landslide.rio.reproject_match(pop)

In [ ]:
print("Reprojected Raster:\n-------------------\n")
print_raster(landslide_repro)
print("Raster to Match:\n----------------\n")
print_raster(pop)

### 1.2. Save reprojected landslide to file

In [ ]:
# Save reprojected landslide data to file
landslide_repro.rio.to_raster("landslide_repro.tif")

### 1.3. Create masked population rasters

In [ ]:
# Loop through risk cats and create one dataset per cat

risk_category_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]

for risk_category in risk_category_list:
    pop_masked = pop.where(landslide_repro == risk_category)
    pop_masked.rio.to_raster('pop_masked_ls_risk_cat_{}.tif'.format(risk_category))

### 1.4. Apply zonal stats on masked population rasters

In [ ]:
# Defining zonal stats function

def ZonalStats(shape_gdf, raster, stats, nodata_value):

    zonalSt = rasterstats.zonal_stats(shape_gdf, raster, stats = stats, nodata=nodata_value)
    
    df = pd.DataFrame(zonalSt)
    
    df_concat = pd.concat([df, shape_gdf], axis=1)
    
    final_gdf = gpd.GeoDataFrame(df_concat, geometry=df_concat.geom) #wkb_geometry
    
    final_gdf_ordered = final_gdf[[c for c in final_gdf if c not in [stats]] + [stats]]
    
    final = final_gdf_ordered.drop(columns=['geometry'])
    
    final = final.set_geometry('geom')
    
    return final

In [ ]:
# Apply zonal stats to each raster

adm_stats = adm
risk_category_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]

for category in risk_category_list:
    raster = 'pop_masked_ls_risk_cat_{}.tif'.format(category)
    print(raster)
    
    adm_stats = ZonalStats(adm_stats, raster, 'sum', 'NaN') #change column name
    
    adm_stats = adm_stats.rename(columns={'sum': 'pop_{}'.format(category)})
    
    col = 'pop_{}'.format(category)
    
    adm_stats[col] = adm_stats[col].round(decimals=2)

In [ ]:
# Create column with pop sum

sum_list = [
    'pop_0',
    'pop_1',
    'pop_2',
    'pop_3',
    'pop_4',
    'pop_5',
    'pop_6',
    'pop_7',
    'pop_8'
]

adm_stats['pop_sum'] = adm_stats[sum_list].sum(axis=1)

### 1.5. Aggregate from 8 to 5 risk categories

In [ ]:
# Aggregated to match desired risk categories
#0=Null; 1-3=Low; 4-5=Moderate; 6=High; 7-9=Very High
adm_stats['pop_ls_0'] = adm_stats['pop_0']
adm_stats['pop_ls_1'] = adm_stats[['pop_1', 'pop_2', 'pop_3']].sum(axis=1)
adm_stats['pop_ls_2'] = adm_stats[['pop_4', 'pop_5']].sum(axis=1)
adm_stats['pop_ls_3'] = adm_stats['pop_6']
adm_stats['pop_ls_4'] = adm_stats[['pop_7', 'pop_8']].sum(axis=1)

## 2. Area


### 2.1. Get pixel size

In [ ]:
# Get km2 per cell

# Reproject to equal area crs
landslide_cea = landslide_repro.rio.reproject('+proj=cea')
#landslide_cea.rio.to_raster("landslide_repro_cea.tif")

res = landslide_cea.rio.resolution()[0]
print('resolution: ', res)

m2_per_cell =res * res
km2_per_cell = m2_per_cell / 1000000

print('m2_per_cell ', m2_per_cell)
print('km2_per_cell ', km2_per_cell)

### 2.2. Get area per risk category per polygon

In [ ]:
# Apply zonal stats with count parameter to each raster
# Multiply count with pixel size to get area

risk_category_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]

for category in risk_category_list:
    raster = 'pop_masked_ls_risk_cat_{}.tif'.format(category)
    print(raster)
    
    adm_stats = ZonalStats(adm_stats, raster, 'count', 0)
 
    adm_stats = adm_stats.rename(columns={'count': 'km2_{}'.format(category)})
    
    col = 'km2_{}'.format(category)
    
    adm_stats[col] = (adm_stats[col] * (87.84301333486803 * 87.84301333486803) / 1000000).round(decimals=2)

In [ ]:
adm_stats

### 2.3. Aggregate from 8 to 5 risk categories


In [ ]:
# Group to match desired categories
#0=Null; 1-3=Low; 4-5=Moderate; 6=High; 7-9=Very High
adm_stats['km2_ls_0'] = adm_stats['km2_0']
adm_stats['km2_ls_1'] = adm_stats[['km2_1', 'km2_2', 'km2_3']].sum(axis=1)
adm_stats['km2_ls_2'] = adm_stats[['km2_4', 'km2_5']].sum(axis=1)
adm_stats['km2_ls_3'] = adm_stats['km2_6']
adm_stats['km2_ls_4'] = adm_stats[['km2_7', 'km2_8']].sum(axis=1)

In [ ]:
adm_stats

## 3. Buildings

In [ ]:
print(datetime.now().strftime("%H:%M:%S"))
# Input: building centroids, landslide dataset
# Output: number of buildings per land slide risk catagory per admin polygon

# Approach:
# Extract pixel value to building centroids
# Loop over landslide risk category
# For each risk category
    # filter the points so the dataset only includes points from that category
    # perform spatial join on remaining points

### 3.1. Visualize data

In [ ]:

#### Explored data
#
#fig, ax = plt.subplots(figsize=(10, 10))
#
#adm.plot(ax=ax)
#landslide_repro.plot(ax=ax)
#build.plot(ax=ax, color='red')
#
#plt.show()

In [ ]:
# Check matching crs
print('adm crs', adm.crs)
print('build crs', build.crs)
print('landslide_repro crs', landslide_repro.rio.crs)

### 3.2. Zonal stats - cell to point

In [ ]:
# Applying point stats and concatenating output to input geometry
# Output: buildings with associated landslide risk column

def point_stats(shape_gdf, raster_file_path):

    pointSt = rasterstats.point_query(shape_gdf, raster_file_path, interpolate='nearest')

    df = pd.DataFrame(pointSt)
    
    df_concat = pd.concat([df, shape_gdf], axis=1)
    
    gdf = gpd.GeoDataFrame(df_concat, geometry=df_concat.geom) #wkb_geometry
    
    gdf_ordered = gdf[[c for c in gdf if c not in [0]] + [0]] # 0 refers to column name
    
    gdf_clean = gdf_ordered.drop(columns=['geometry']).set_geometry('geom').rename(columns={0:'risk'})
        
    return gdf_clean

In [ ]:
print(datetime.now().strftime("%H:%M:%S"))
build_risk = point_stats(build, 'landslide_repro.tif')

### 3.3. Spatial join and grouping

In [ ]:
print(datetime.now().strftime("%H:%M:%S"))
# Joining points to polygon
# Group points by pcode and get count (=number of points within each pcode)
# Merge point back to original polygon dataset

# Group by parameter = pcode

def buildings(polygons, build_centroids, group_by_parameter, build_count_column):
    
    # Join building centroids to flood polygons
    #print('    Joining buildings to flood polygons   Start: {}'.format(datetime.now().strftime("%H:%M:%S")))
    joined_df = gpd.sjoin(
        build_centroids,
        polygons,
        how='inner',
        predicate='intersects'
    )
    #print('    Joining buildings to flood polygons   End:   {}'.format(datetime.now().strftime("%H:%M:%S")))
    
    # Count number of buildings within admin polygons
    #print('    Counting number of buildings          Start: {}'.format(datetime.now().strftime("%H:%M:%S")))
    build_count = joined_df.groupby(
        [group_by_parameter],
        as_index=False,
    )['geom'].count() # vuid is arbitrary, we just count the row
    #print('    Counting number of buildings          End:   {}'.format(datetime.now().strftime("%H:%M:%S")))
    
    # Change column name to build_count
    build_count.rename(columns = {'geom': build_count_column}, inplace = True)

    # Merge build count back on to admin dataset
    polygons = polygons.merge(
        build_count, 
        on=group_by_parameter, 
        how='left')
    
    return polygons

In [ ]:
print(datetime.now().strftime("%H:%M:%S"))
# loop over risk categories

risk_category_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]

for category in risk_category_list:
    
    build_filtered = build_risk.query('risk == {}'.format(category))
    
    build_count_column = 'build_{}'.format(category)
    
    adm_stats = buildings(adm_stats, build_filtered, pcode, build_count_column)

### 3.4. Aggregate from 8 to 5 risk categories

In [ ]:
# Group to match desired categories
#0=Null; 1-3=Low; 4-5=Moderate; 6=High; 7-9=Very High
adm_stats['build_ls_0'] = adm_stats['build_0']
adm_stats['build_ls_1'] = adm_stats[['build_1', 'build_2', 'build_3']].sum(axis=1)
adm_stats['build_ls_2'] = adm_stats[['build_4', 'build_5']].sum(axis=1)
adm_stats['build_ls_3'] = adm_stats['build_6']
adm_stats['build_ls_4'] = adm_stats[['build_7', 'build_8']].sum(axis=1)

In [ ]:
adm_stats

### 4. Clean and join to original adm_stats

In [ ]:
adm_stats.to_postgis('region_stats_landslide_v2', con, if_exists='replace')

In [ ]:
## Select columns to join
#cols_to_use = adm_stats.columns.difference(adm_stats_original.columns).to_list()
#cols_to_use.append(pcode)
#
## Join selected columns to original stats table
#merged = adm_stats_original.merge(
#                    adm_stats[cols_to_use],
#                    on=pcode,
#                    how='left').fillna(0)
#
## Save original+join to local db for qa
#merged.to_postgis('merged_landslide', con, if_exists='replace')
#
#merged

### 5. Post-processing

In [ ]:
# Load recent outputs
adm2 = gpd.GeoDataFrame.from_postgis('SELECT * from adm2_stats_landslide', con) #.to_crs(repro_crs)
adm1 = gpd.GeoDataFrame.from_postgis('SELECT * from adm_stats_landslide', con)
#adm_region = gpd.GeoDataFrame.from_postgis('SELECT * from region_stats_landslide', con)

list_of_data = [region_stats_landslide_v2] # [adm2, adm1]#, adm_region]
list_of_table_names = ['afg_admbnda_region_stats_v2'] #['afg_admbnda_adm2_stats_v2', 'afg_admbnda_adm1_stats_v2', 'afg_admbnda_region_stats_v2']

In [ ]:
# List columns to drop
drop_cols = ['pop_0',
             'pop_1',
             'pop_2',
             'pop_3',
             'pop_4',
             'pop_5',
             'pop_6',
             'pop_7',
             'pop_8',
             'pop_sum',
             'km2_0',
             'km2_1',
             'km2_2',
             'km2_3',
             'km2_4',
             'km2_5',
             'km2_6',
             'km2_7',
             'km2_8',
             'build_0',
             'build_1',
             'build_2',
             'build_3',
             'build_4',
             'build_5',
             'build_6',
             'build_7',
             'build_8']

# List columns to round to integer
round_cols = [
    'pop',
    'build',
    'pop_f029',
    'build_f029',
    'pop_f121',
    'build_f121',
    'pop_f271',
    'build_f271',
    'pop_av',
    'build_av',
    'pop_ls_0',
    'pop_ls_1',
    'pop_ls_2',
    'pop_ls_3',
    'pop_ls_4',
    'build_ls_0',
    'build_ls_1',
    'build_ls_2',
    'build_ls_3',
    'build_ls_4'
]

In [ ]:
#for i in range(len(list_of_data)):
#    
#    adm = list_of_data[i]
#    table_name = list_of_table_names[i]
#    
#    # Resetting risk category 0
#    # Set risk category 0 to original pop - sum(risk category 1-4)
#    adm['pop_ls_0'] = adm.loc[:,'pop'] - adm.loc[:,['pop_ls_1', 'pop_ls_2', 'pop_ls_3', 'pop_ls_4']].sum(axis=1) #.round(decimals=2)
#    adm['km2_ls_0'] = adm.loc[:,'km2'] - adm.loc[:,['km2_ls_1', 'km2_ls_2', 'km2_ls_3', 'km2_ls_4']].sum(axis=1) #.round(decimals=2)
#    adm['build_ls_0'] = adm.loc[:,'build'] - adm.loc[:,['build_ls_1', 'build_ls_2', 'build_ls_3', 'build_ls_4']].sum(axis=1) #.round(decimals=2)
#
#    # Drop cols
#    adm = adm.drop(columns=drop_cols)
#
#    # Move geom column to end of table
#    cols_at_end = ['geom']
#    adm = adm[[c for c in adm if c not in cols_at_end] 
#            + [c for c in cols_at_end if c in adm]]
#
#    # Rename geom column
#    #adm = adm.rename(columns={'geom': 'geometry'}).set_geometry('geometry')
#
#    # Round all stats to 2 decimal points
#    adm.loc[:,'pop':] = adm.loc[:,'pop':].round(2)
#    adm.loc[:, round_cols] = adm.loc[:, round_cols].fillna(0).astype('int')
#
#    adm.to_postgis(table_name, con, if_exists='replace')

In [ ]:
#adm2 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2_stats_delete', con) #.to_crs(repro_crs)
#adm1 = gpd.GeoDataFrame.from_postgis('SELECT * from adm_stats_landslide', con)
#adm_region = gpd.GeoDataFrame.from_postgis('SELECT * from region_stats_landslide', con)


In [9]:
reg = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_region_stats_v2', con, 'geometry')

In [15]:
reg = reg.rename(columns={'geometry': 'geom'}).set_geometry('geom')

In [16]:
# Move geom column to end of table
cols_at_end = ['geom']
reg = reg[[c for c in reg if c not in cols_at_end] 
        + [c for c in cols_at_end if c in reg]]

In [17]:
reg

,tid,id,reg_en,reg_da,reg_pcode,reg_ref,regalt1en,regalt2en,regalt1da,regalt2da,adm0_en,adm0_da,adm0_pcode,date,validon,validto,shape_leng,shape_area,pop,km2,sett,build,pop_density,pop_f029,km2_f029,build_f029,pop_f121,km2_f121,build_f121,pop_f271,km2_f271,build_f271,km2_av,pop_av,build_av,pop_ls_0,pop_ls_1,pop_ls_2,pop_ls_3,pop_ls_4,km2_ls_0,km2_ls_1,km2_ls_2,km2_ls_3,km2_ls_4,build_ls_0,build_ls_1,build_ls_2,build_ls_3,build_ls_4,geom
0,1,1,Northern,سهیلی حوزه,NR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.678772,7.751351,5235170,77306.89,4714,1112341,67.72,641151,6401.16,226165,352382,3833.59,109747,123184,1318.61,33472,2982.31,95657,18817,3741116,1688,748423,687021,56920,42039.43,79.15,20452.62,13214.08,1521.61,1004581,92,57712,49139,817,"POLYGON ((68.43099 36.07082, 68.42871 36.06806..."
1,2,2,Central Highland,لوړه مرکزی حوزه,CH,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.035766,3.100092,1224738,31641.50,4001,141838,38.71,48397,857.81,12470,35019,602.58,7939,16340,246.78,3357,3382.76,116303,17630,249666,58983,539781,271531,104775,3591.54,1473.89,16428.51,7309.05,2838.51,66744,8211,53643,12107,1133,"POLYGON ((65.61898 33.31015, 65.61822 33.30971..."
2,3,3,South Eastern,جنوب ختیځه حوزه,SE,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,15.949378,4.825809,3572212,50019.50,6882,596874,71.42,627562,6432.82,148378,377457,3987.60,73486,129614,1328.43,15912,1213.01,78842,3522,2371694,120196,585189,385661,109470,29363.46,2042.84,10686.53,6515.44,1411.23,557210,8063,16681,14778,142,"POLYGON ((68.17123 31.83539, 68.17001 31.83832..."
3,4,4,Southern,جنوبی حوزه,SR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,27.822820,17.576351,4322541,185698.95,7815,1190168,23.28,1122231,32422.42,362885,681793,16703.33,197798,268623,5295.10,60643,3774.49,85639,4739,3653686,94724,359413,185635,29081,154314.40,4401.68,15232.72,10359.61,1390.54,1162827,4962,18148,4169,62,"POLYGON ((67.58714 31.50558, 67.58714 31.50522..."
4,5,5,Eastern,ختیځه حوزه,ER,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,10.390809,2.472625,3847087,25074.13,3555,488660,153.43,570548,1354.15,85563,333644,887.75,39404,172397,390.39,15698,2542.40,126115,3832,2359854,256418,707653,350390,172770,4126.37,3199.39,9745.48,4610.47,3392.42,437649,9825,25989,14240,957,"POLYGON ((70.00272 34.04593, 70.00179 34.04587..."
5,6,6,Capital,مرکزی حوزه,CR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,14.491308,3.054274,9377610,31086.15,6007,1023867,301.67,2286357,1739.51,245411,1570888,1048.60,146535,795946,415.95,66473,2986.34,305379,26367,7325615,118773,1164957,534098,234165,8471.06,2117.39,10432.25,6888.02,3177.43,896984,5777,90877,28529,1700,"POLYGON ((67.27185 33.97512, 67.26954 33.97330..."
6,7,7,North Eastern,سهیل ختیځه حوزه,NE,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,27.870669,8.141572,5463033,80700.71,6053,995154,67.69,680430,4765.26,169647,446400,3109.97,86610,207903,1227.98,28374,10800.06,402390,33810,3038725,142478,1111029,845455,325343,14775.73,5700.14,28985.39,20442.93,10796.52,830930,10513,82686,68364,2661,"POLYGON ((71.19470 36.03890, 71.19362 36.03821..."
7,8,8,Western,لویدیځه حوزه,WR,None,None,None,None,None,Afghanistan,افغانستان,AF,2017-12-12,2021-11-17,None,26.348523,15.642130,5779766,160616.85,7817,1038744,35.98,1078627,19018.18,256281,649972,11002.06,132366,262553,3860.51,34453,6070.83,117627,20149,4156823,77757,962082,484038,99065,91753.50,3862.81,43680.86,15970.54,5349.14,915511,4527,85662,32131,913,"POLYGON ((61.61755 31.38748, 61.60269 31.38935..."


In [19]:
reg.to_postgis('afg_admbnda_region_stats_v2', con, if_exists='replace')